In [48]:
#creazione split
#feat_train, feat_test, target_train, target_test = train_test_split(X, y, test_size=0.20, random_state=28) 
#smote = SMOTENC(sampling_strategy='dict', categorical_features=['bkl', 'vasc'])
#X_sm, y_sm = smote.fit_sample(X,y)
#len(X)
#len(X_sm)


#print(type(X))

In [98]:
# imports
import os
import cv2
from sklearn.ensemble import BaggingClassifier
#import matplotlib.pyplot as plt
#from sklearn.model_selection import train_test_split
#import sklearn as skl
import sklearn.preprocessing as preproc
import numpy as np
import keras as k
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D, Flatten, Dense, Dropout, MaxPool2D
#import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
#from time import time
import pandas as pd
from imblearn.over_sampling import SMOTENC
import tensorflow as tf
from PIL import Image

In [109]:
#### variabili globali
#### GIUSEPPE
image_path = '..\skin-cancer-mnist-ham10000\HAM10000_images_part_1'
#### FABIO
# image_path = '..\skin-cancer-mnist-ham10000\HAM10000_images_part_1'
csv_path = 'HAM10000_metadata.csv'
dataset = pd.read_csv(csv_path, encoding = "ISO-8859-1")
labels = set(dataset["dx"])
cat_column=["dx"]
ohe = preproc.OneHotEncoder(sparse=False, handle_unknown="ignore") 

In [106]:
######## API varie
# - carica in memoria immagini e label
# - 1-hot encoding
# - implementazione modello tramite ensamble (nel classificatore va incluso il numero di labels)
# - train e test usando il bagging, ma prima bisogna fare SMOTE su vasc e bkl?



# carica tutto in un nuovo dataframe
def load_pictures(path, dataset):
    col=["lesion_id" ,"image_id","dx","dx_type" ,"age" ,"sex" ,"localization","path","pixels"]
    dataset.insert(7,"path",None)
    dataset.insert(8,"pixels",None)
    data=pd.DataFrame(columns=col)
    helper = pd.DataFrame(columns=col)
    for _,_,files in os.walk(path+'\\'):
        for file in files: 
            file_replace = file.replace(".jpg", "") #tagliare il .jpg che non compare nel dataset
            path2=path+"\\"+file #path completo di un file
            helper=dataset.loc[dataset["image_id"]==file_replace] #tupla del file
            helper["path"]=path2
            col=["lesion_id" ,"image_id","dx","dx_type" ,"age" ,"sex" ,"localization"]
            data = data.append(helper)
    data["pixels"]=data['path'].map(lambda x: np.asarray(cv2.imread(x))) #associazione ai pixel
    return data


def lab_encode(dataset):#Ho una sola colonna! 
    for col in cat_column:
        new_le = preproc.LabelEncoder()
        new_ds = dataset
        new_ds[col] = new_le.fit_transform(dataset[col]) 
      #  label_encoders[col] = new_le 
    return new_ds

def onehot_encode(dataset):
    dataset_LabEncoded = lab_encode(dataset)
    x=np.array([dataset_LabEncoded["dx"]]) 
    y_onehot = ohe.fit_transform(x.reshape(-1,1))
    return y_onehot


# si prende tutto l'array di risultati
def reverse_onehot(vector):
    res = ohe.inverse_transform(vector)
    return res


# conteggio statistiche dataset
def stats(dataset):
    labels = set(dataset['dx'])
    for label in labels:
        print(str(label), ':\t', str(dataset[dataset['dx']==label]['dx'].count()))
        #print (label)

        
        
def new_classifier():
    input_img = Input(shape=(450, 600, 3))  # 3x600x450 image RGB 
    #print (tf.shape(input_img))
    x = Conv2D(20, (5, 5), activation='relu', padding='same')(input_img) # 20x450x600
    #print (tf.shape(x))
    x = MaxPooling2D((2, 2), padding='same')(x) # 20x225x300
    #print (tf.shape(x))
    x = Conv2D(140, (3, 3), activation='relu', padding='same')(x) # 140x75x100
    #print (tf.shape(x))
    x = MaxPooling2D((2, 2), padding='same')(x) # 40x38x50
    #print (tf.shape(x))
    x = Conv2D(50, (3, 3), activation='relu', padding='same')(x) # 50x38x50
    #print (tf.shape(x))
    x = MaxPooling2D((2, 2), padding='same')(x) # 50x19x25
    #print (tf.shape(x))
    x = Flatten()(x) # qui la x diventa monodimensionale
    #print (tf.shape(x))
    # da qualche parte ci va il numero delle label?
    encoded = Dense(7, activation='softmax')(x)
    classifier = k.models.Model(input_img, encoded)   #questo è il nostro base_estimator, compile configura per il training
    classifier.compile(optimizer='adadelta', loss='binary_crossentropy')
    return classifier

def new_classifier2():
    # Set the CNN model 
    input_shape = (450, 600, 3)
    num_classes = 7
    
    model = k.models.Sequential()
    model.add(Conv2D(20, kernel_size=(5, 5),activation='relu',padding = 'Same', input_shape = input_shape))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Conv2D(140,kernel_size=(3, 3), activation='relu',padding = 'Same'))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Conv2D(50,kernel_size=(3, 3), activation='relu',padding = 'Same'))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.summary()
    model.compile(optimizer='adadelta', loss='binary_crossentropy')
    return model


def ensemble():
    #da settare 
    #    base_estimator il classificatore come CNN,
    #    n_estimators,
    #    max_samples è il numero di elementi nei training set,
    #    bootstrap se lo vogliamo o no (booleano),
    #    n_jobs per il parallelismo,
    classifier = new_classifier2()
    bagging = BaggingClassifier(base_estimator=classifier, n_estimators=1, 
                                bootstrap=True, n_jobs=4)
    return bagging

In [41]:
#creazione dataframe
ds = load_pictures(image_path, dataset)

C:\Users\jph94\Anaconda3\envs\keras_env\lib\site-packages\ipykernel_launcher.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [108]:
# encoding delle labels (dx)
encoded = lab_encode(ds)

In [107]:
# da provare
X = encoded["pixels"]
y = encoded["dx"]
bag_class = ensemble()
X = np.asarray(X)
y = np.asarray(y)

dataset_size = len(X)
reshaped = np.reshape(X,(dataset_size,-1))
params = bag_class.get_params()
bag_class.fit(reshaped, y)


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 450, 600, 20)      1520      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 225, 300, 20)      0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 225, 300, 140)     25340     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 112, 150, 140)     0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 112, 150, 50)      63050     
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 56, 75, 50)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 56, 75, 50)       

TypeError: Cannot clone object '<keras.engine.sequential.Sequential object at 0x0000021DD97460F0>' (type <class 'keras.engine.sequential.Sequential'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' methods.